### Implementing FunkSVD

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# Create data subset
user_movie_subset = user_by_movie[[75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[  7.  10.   8.]
 [  6.  10.   7.]
 [  8.   9.   8.]
 [  8.  10.  10.]
 [  9.   9.   9.]
 [  8.   9.   9.]]


In [3]:
len(user_movie_subset)

6

`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the funtion on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of matrix factorization.

In [8]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0]# number of rows in the matrix
    n_movies = ratings_mat.shape[1]# number of movies in the matrix
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat))# total number of ratings in the matrix
    
    # initialize the user and movie matrices with random values
    # helpful link: https://numpy.org/doc/stable/reference/random/generated/numpy.random.rand.html
    user_mat = np.random.rand(n_users,latent_features)# user matrix filled with random values of shape user x latent
    movie_mat = np.random.rand(latent_features,n_movies)# movie matrix filled with random values of shape latent x movies
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # keep track of iteration and MSE
    print("Optimizaiton Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for iteration in range(iters):
        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):       
                # if the rating exists
                if np.isnan(ratings_mat[i,j]) is False:        
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    sse = ratings_mat[i,j] - np.dot(user_mat[i, :], movie_mat[:,j])
                    # Keep track of the sum of squared errors for the matrix
                    sse_accum += sse**2
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[i, k] += learning_rate * (2*sse*movie_mat[k, j])
                        movie_mat[k, j] += learning_rate * (2*sse*user_mat[i, k])
        
        # print results
        print("%d \t\t %f" % (iteration+1, sse_accum / num_ratings))    
    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 3 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [9]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=3, learning_rate=0.005, iters=10)# use your function with 3 latent features, lr of 0.005 and 10 iterations

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 0.000000
2 		 0.000000
3 		 0.000000
4 		 0.000000
5 		 0.000000
6 		 0.000000
7 		 0.000000
8 		 0.000000
9 		 0.000000
10 		 0.000000


In [10]:
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 0.33580126  0.40853473  0.37658994]
 [ 0.58727587  0.56557677  0.62089167]
 [ 1.23021671  0.66479206  1.16492101]
 [ 0.78290642  0.35915351  0.677453  ]
 [ 0.85193223  0.65146757  0.78089998]
 [ 1.00796111  0.35976924  0.96885246]]
[[  7.  10.   8.]
 [  6.  10.   7.]
 [  8.   9.   8.]
 [  8.  10.  10.]
 [  9.   9.   9.]
 [  8.   9.   9.]]


_**Task: compare the predicted vs actual ratings and write your findings here**_

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 3 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 300.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about the `mean squared error` at the end of each training iteration?

In [11]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=3, learning_rate=0.005, iters=300)#use your function with 3 latent features, lr of 0.005 and 300 iterations

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 0.000000
2 		 0.000000
3 		 0.000000
4 		 0.000000
5 		 0.000000
6 		 0.000000
7 		 0.000000
8 		 0.000000
9 		 0.000000
10 		 0.000000
11 		 0.000000
12 		 0.000000
13 		 0.000000
14 		 0.000000
15 		 0.000000
16 		 0.000000
17 		 0.000000
18 		 0.000000
19 		 0.000000
20 		 0.000000
21 		 0.000000
22 		 0.000000
23 		 0.000000
24 		 0.000000
25 		 0.000000
26 		 0.000000
27 		 0.000000
28 		 0.000000
29 		 0.000000
30 		 0.000000
31 		 0.000000
32 		 0.000000
33 		 0.000000
34 		 0.000000
35 		 0.000000
36 		 0.000000
37 		 0.000000
38 		 0.000000
39 		 0.000000
40 		 0.000000
41 		 0.000000
42 		 0.000000
43 		 0.000000
44 		 0.000000
45 		 0.000000
46 		 0.000000
47 		 0.000000
48 		 0.000000
49 		 0.000000
50 		 0.000000
51 		 0.000000
52 		 0.000000
53 		 0.000000
54 		 0.000000
55 		 0.000000
56 		 0.000000
57 		 0.000000
58 		 0.000000
59 		 0.000000
60 		 0.000000
61 		 0.000000
62 		 0.000000
63 		 0.000000
64 		 0

In [12]:
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 0.03729612  0.04153088  0.04935605]
 [ 0.29329264  0.40856855  0.27851271]
 [ 0.56201048  0.66450193  0.76272784]
 [ 0.17129719  0.27252948  0.10423976]
 [ 0.36565193  0.35534617  0.56020161]
 [ 0.1513715   0.09144756  0.31889993]]
[[  7.  10.   8.]
 [  6.  10.   7.]
 [  8.   9.   8.]
 [  8.  10.  10.]
 [  9.   9.   9.]
 [  8.   9.   9.]]


_**Task: compare the predicted vs actual ratings and write your findings here**_

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 3 latent features, a learning rate of 0.005, and 450 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [13]:
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[ nan,  10.,   8.],
        [  6.,  10.,   7.],
        [  8.,   9.,   8.],
        [  8.,  10.,  10.],
        [  9.,   9.,   9.],
        [  8.,   9.,   9.]])

In [14]:
# run SVD on the matrix with the missing value
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=3, learning_rate=0.005, iters=450)
#use your function with 3 latent features, lr of 0.005 and 450 iterations

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 0.000000
2 		 0.000000
3 		 0.000000
4 		 0.000000
5 		 0.000000
6 		 0.000000
7 		 0.000000
8 		 0.000000
9 		 0.000000
10 		 0.000000
11 		 0.000000
12 		 0.000000
13 		 0.000000
14 		 0.000000
15 		 0.000000
16 		 0.000000
17 		 0.000000
18 		 0.000000
19 		 0.000000
20 		 0.000000
21 		 0.000000
22 		 0.000000
23 		 0.000000
24 		 0.000000
25 		 0.000000
26 		 0.000000
27 		 0.000000
28 		 0.000000
29 		 0.000000
30 		 0.000000
31 		 0.000000
32 		 0.000000
33 		 0.000000
34 		 0.000000
35 		 0.000000
36 		 0.000000
37 		 0.000000
38 		 0.000000
39 		 0.000000
40 		 0.000000
41 		 0.000000
42 		 0.000000
43 		 0.000000
44 		 0.000000
45 		 0.000000
46 		 0.000000
47 		 0.000000
48 		 0.000000
49 		 0.000000
50 		 0.000000
51 		 0.000000
52 		 0.000000
53 		 0.000000
54 		 0.000000
55 		 0.000000
56 		 0.000000
57 		 0.000000
58 		 0.000000
59 		 0.000000
60 		 0.000000
61 		 0.000000
62 		 0.000000
63 		 0.000000
64 		 0

In [15]:
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is: {}".format(preds[0,0]))
print()
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 7. Not bad!")

The predicted value for the missing rating is: 0.43662530603500427

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 7. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 1000 users.  In the above portion, you were using a very small subset of data with no missing values.

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 3 latent features, 0.005 learning rate, and 500 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [16]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=3, learning_rate=0.005, iters=500)
#fit to 1000 users with 3 latent features, lr of 0.005, and 500 iterations

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 0.000000
2 		 0.000000
3 		 0.000000
4 		 0.000000
5 		 0.000000
6 		 0.000000
7 		 0.000000
8 		 0.000000
9 		 0.000000
10 		 0.000000
11 		 0.000000
12 		 0.000000
13 		 0.000000
14 		 0.000000
15 		 0.000000
16 		 0.000000
17 		 0.000000
18 		 0.000000
19 		 0.000000
20 		 0.000000
21 		 0.000000
22 		 0.000000
23 		 0.000000
24 		 0.000000
25 		 0.000000
26 		 0.000000
27 		 0.000000
28 		 0.000000
29 		 0.000000
30 		 0.000000
31 		 0.000000
32 		 0.000000
33 		 0.000000
34 		 0.000000
35 		 0.000000
36 		 0.000000
37 		 0.000000
38 		 0.000000
39 		 0.000000
40 		 0.000000
41 		 0.000000
42 		 0.000000
43 		 0.000000
44 		 0.000000
45 		 0.000000
46 		 0.000000
47 		 0.000000
48 		 0.000000
49 		 0.000000
50 		 0.000000
51 		 0.000000
52 		 0.000000
53 		 0.000000
54 		 0.000000
55 		 0.000000
56 		 0.000000
57 		 0.000000
58 		 0.000000
59 		 0.000000
60 		 0.000000
61 		 0.000000
62 		 0.000000
63 		 0.000000
64 		 0

`6.` Now that you have a set of predictions for each user-movie pair.  Let's answer a few questions about your results. Provide the correct values to each of the variables below, and check your solutions using the tests below.

In [17]:
# How many actual ratings exist in first_1000_users
num_ratings = np.count_nonzero(~np.isnan(first_1000_users))
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()

# How many ratings did we make for user-movie pairs that didn't have ratings
ratings_for_missing = first_1000_users.shape[0]*first_1000_users.shape[1] - num_ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

The number of actual ratings in the first_1000_users is 14287.

The number of ratings made for user-movie pairs that didn't have ratings is 13835713


In [18]:
# Test your results against the solution
assert num_ratings == 14287, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 13835713, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")

Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?
